# Revised case normalization for Hirslanden Aarau 2018

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys

import pandas as pd
import numpy as np

from loguru import logger

sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize, remove_leading_zeros
from src.service import bfs_cases_db_service as bfs_db
from src.service.bfs_cases_db_service import session, get_sociodemographics_for_hospital_year, get_earliest_revisions_for_aimedic_ids, get_codes, apply_revisions
from src.revised_case_normalization.py.format_for_grouper import format_for_grouper

In [2]:
file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
        'Hirslanden Aarau',
        '2018',
        ['Aarau 2018'])

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2018', sheets=['Aarau 2018'])


In [3]:
df_revised_case_d2d = normalize(file_info, 0)


2022-10-31 09:24:31.690 | INFO     | src.revised_case_normalization.py.normalize:normalize:35 - Read 17 cases for Hirslanden Aarau 2018
2022-10-31 09:24:31.693 | INFO     | src.revised_case_normalization.py.normalize:normalize:57 - TYPES:
tranche                   string
datum                     string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung möglich       string
cw

In [4]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041623461,0001059029,W,77,2,41623461,I7024,I7024,M100,3,F59E,[J4481],[J4483],[],[]
1,0041665678,0002043896,M,82,17,41665678,I5001,I5001,M100,3,F62C,[R18],[],[],[]
2,0041698586,0022177655,M,93,2,41698586,I7024,I7024,M100,3,F59E,[I743],[],[],[]
3,0041719219,0001339311,M,91,4,41719219,I495,I495,M100,3,F71B,[N182],[],[],[]
4,41777951,0022177655,M,93,2,41777951,I7024,I7024,M100,0,F59E,"[N184, T828]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [5]:
cases_in_db = get_sociodemographics_for_hospital_year(file_info.hospital_name_db, int(file_info.year))
cases_in_db[NORM_CASE_ID_COL] = cases_in_db[CASE_ID_COL].apply(remove_leading_zeros)
print(cases_in_db.head())

2022-10-31 09:24:32.543 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:124 - Read 10112 rows from the DB, for the hospital 'Hirslanden Aarau' in 2018


   aimedic_id     case_id        patient_id  age_years  age_days  admission_weight  gestation_age gender admission_date grouper_admission_type discharge_date grouper_discharge_type  duration_of_stay  ventilation_hours case_id_norm
0      115452  0041415230  A0E36E42C141D541         29         0                 0              0      M     2018-01-16                     01     2018-01-18                     00                 2                  0     41415230
1      115453  0041453333  10CA4E4A4D6A6648         34         0                 0              0      W     2018-01-19                     01     2018-01-20                     00                 1                  0     41453333
2      115454  0041473239  E7A902368D82844A         63         0                 0              0      M     2018-03-19                     01     2018-03-23                     00                 4                  0     41473239
3      115455  0041506268  5FCEADE16E4FD6EC         44         0            

In [6]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)

cols_to_join.remove(PATIENT_ID_COL)

# Merge cases in db with the revised cases

joined = pd.merge(df_revised_case_d2d, cases_in_db,
                  how='left',
                  on=cols_to_join,
                  suffixes=('', '_db'))

joined

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops,aimedic_id,case_id_db,patient_id_db,age_days,admission_weight,gestation_age,admission_date,grouper_admission_type,discharge_date,grouper_discharge_type,ventilation_hours
0,0041623461,0001059029,W,77,2,41623461,I7024,I7024,M100,3,F59E,[J4481],[J4483],[],[],115875,0041623461,A9832A23F251C511,0,0,0,2018-01-10,01,2018-01-12,00,0
1,0041665678,0002043896,M,82,17,41665678,I5001,I5001,M100,3,F62C,[R18],[],[],[],116851,0041665678,2C4C8942514B875C,0,0,0,2018-01-30,01,2018-02-16,00,0
2,0041698586,0022177655,M,93,2,41698586,I7024,I7024,M100,3,F59E,[I743],[],[],[],117722,0041698586,296ABD4577F82DE5,0,0,0,2018-03-15,01,2018-03-17,00,0
3,0041719219,0001339311,M,91,4,41719219,I495,I495,M100,3,F71B,[N182],[],[],[],118215,0041719219,F99B9F90EF929909,0,0,0,2018-03-17,01,2018-03-21,00,0
4,41777951,0022177655,M,93,2,41777951,I7024,I7024,M100,0,F59E,"[N184, T828]",[],[],[],120279,0041777951,296ABD4577F82DE5,0,0,0,2018-05-31,01,2018-06-02,00,0
5,0041648887,0001709901,M,64,8,41648887,I5014,I5014,M100,3,F62C,[E1191],[E1190],[],[],116472,0041648887,E04F10AADF78511A,0,0,0,2018-01-17,01,2018-01-25,00,8
6,0041709615,0001749027,W,58,5,41709615,N390,N390,M100,3,L63D,[B370],[],[],[],117999,0041709615,7FFA5616E43C0E6F,0,0,0,2018-03-09,01,2018-03-14,00,0
7,0041772407,0022190089,W,79,9,41772407,N390,N390,M100,3,L63D,[E440],[],[890A32],[],120078,0041772407,63E614CAFA7A0DCF,0,0,0,2018-05-23,01,2018-06-01,00,0
8,0041769953,0022179314,M,70,6,41769953,N1781,N1781,M100,3,L60D,[J9610],[],[],[],119991,0041769953,E16B947907F36775,0,0,0,2018-05-19,01,2018-05-25,00,0
9,0041803251,0001720953,M,51,8,41803251,I208,I208,M200,3,F06D,[J9580],[],[],[],121288,0041803251,9BC945841F642D0A,0,0,0,2018-07-04,01,2018-07-12,00,8


In [7]:
# Print out how many rows could not be matched
unmatched = joined[joined[AIMEDIC_ID_COL].isna()]
num_unmatched = unmatched.shape[0]
logger.info(f'{num_unmatched} rows could not be matched, given {sorted(cols_to_join)}')

2022-10-31 09:24:32.584 | INFO     | __main__:<module>:4 - 0 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


In [8]:
original_revision_ids = get_earliest_revisions_for_aimedic_ids(joined[AIMEDIC_ID_COL].values.tolist())
original_cases = get_codes(original_revision_ids)
original_cases

,aimedic_id,revision_id,old_pd,secondary_diagnoses,primary_procedure,secondary_procedures
0,115875,115875,I7024,"[I7020, Z9588, I1090, N184, J4483, ...]",395014,"[395011, 397510, 004C12, 005599, 0043, ...]"
1,116472,116472,I5014,"[I5001, I2513, I2522, N184, I1320, ...]",99B712,"[8945, 860A06, 948X40]"
2,116851,116851,I5001,"[J91, I5013, I7020, L304, N185, ...]",887211,"[887963, 887964, 887975, 3491]"
3,117722,117722,I7024,"[Z9588, N184, Z922]",395011,"[397511, 395021, 397510, 0042, 004B18, ...]"
4,117999,117999,N390,"[B965, R400, G3520, F446, Z936]",992217,[]
5,118215,118215,I495,"[I484, I480, I1090, I2519, I340, ...]",893909,[]
6,119991,119991,N1781,"[Y579, I1100, I480, I5001, G2010, ...]",992909,[]
7,120078,120078,N390,"[B962, A4158, R650, E1190, E788, ...]",992217,[]
8,120279,120279,I7024,"[Z9588, Z921, Z922]",395014,"[004C12, 395011, 0043, 004B1A, 004B46, ...]"
9,121288,121288,I208,"[I2513, D684, U6912, E788]",361226,"[361121, 361122, 361C14, 3963, 3964, ...]"


In [9]:
# Format joined dataset to the SwissDRG Batchgrouper 2017 Format 

grouper_input_data_string = format_for_grouper(joined)

grouper_input_data_string

grouper_input_data_string[17]

2022-10-31 09:24:33.263 | INFO     | src.revised_case_normalization.py.format_for_grouper:format_for_grouper:91 - Formatted 17 cases


KeyError: 17

In [ ]:
import subprocess


example_batch_line = """0044007489;57;0;0|0;W;20190101;01;20190101;00;1;0;C20|Z432|N40|I440|I493;465110::20190101|4823::20190101|009A13::20190101;"""

# add date to procedures
# change ; with | at one spot
test_df_line = """17722;93;;;M;20180315;01;20180317;00;2;0;I7024|Z9588|N184|Z922|I743;395011::|397511::|395021::|397510::|0042::|004B18::|004B1A::|884911::|005599::|0046::|393012::;"""

grouper_result = subprocess.check_output(["java",
                                "-cp",
                                "/home/jovyan/work/resources/aimedic-grouper-assembly-0.0.0-SNAPSHOT.jar",
                                "ch.aimedic.grouper.BatchGroupeOne",
                                grouper_input_data_string[6]
                                #test_df_line,
                                ]).decode("utf-8")


grouperResults = subprocess.check_output(["java",
                                "-cp",
                                "/home/jovyan/work/resources/aimedic-grouper-assembly-0.0.0-SNAPSHOT.jar",
                                "ch.aimedic.grouper.BatchGroupMany",
                                f'{grouper_input_data_string[1]}#{grouper_input_data_string[6]}#{grouper_input_data_string[16]}',
                                "#"
                                #test_df_line,
                                ]).decode("utf-8")

print(grouper_result)

In [ ]:
# TODO:
# - Add medication 
# - Add and compare CHOP Codes for sideness and procedure date